In [49]:
import torch
import transformers
transformers.__version__

'4.57.1'

In [ ]:
from Biatron.config import BiatronConfig
from Biatron.modeling_biatron import BiatronForCausalLM


model = BiatronForCausalLM(BiatronConfig())
model

BiatronForCausalLM(
  (model): BiatronModel(
    (embed_tokens): Embedding(32000, 960, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x BiatronDecoderLayer(
        (self_attn): BiatronAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=True)
          (k_proj): Linear(in_features=960, out_features=320, bias=True)
          (v_proj): Linear(in_features=960, out_features=320, bias=True)
          (o_proj): Linear(in_features=960, out_features=960, bias=True)
        )
        (mlp): BiatronMLP(
          (up_proj): Linear(in_features=960, out_features=3840, bias=True)
          (down_proj): Linear(in_features=3840, out_features=960, bias=True)
        )
        (input_layernorm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
      )
    )
    (norm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
    (rotary_emb): BiatronRotaryEmbedding()
  )
  (l

In [54]:
from utils_convert import load_state_dict_biatron

In [55]:
model = load_state_dict_biatron(model, "/home/dan/CEIA/MEGATRON/convert/dict_model.pth")

In [56]:
model

BiatronForCausalLM(
  (model): BiatronModel(
    (embed_tokens): Embedding(32000, 960, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x BiatronDecoderLayer(
        (self_attn): BiatronAttention(
          (q_proj): Linear(in_features=960, out_features=960, bias=True)
          (k_proj): Linear(in_features=960, out_features=320, bias=True)
          (v_proj): Linear(in_features=960, out_features=320, bias=True)
          (o_proj): Linear(in_features=960, out_features=960, bias=True)
        )
        (mlp): BiatronMLP(
          (up_proj): Linear(in_features=960, out_features=3840, bias=True)
          (down_proj): Linear(in_features=3840, out_features=960, bias=True)
        )
        (input_layernorm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
      )
    )
    (norm): LayerNorm((960,), eps=1e-05, elementwise_affine=True)
    (rotary_emb): BiatronRotaryEmbedding()
  )
  (l

In [57]:
input_ids = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], device='cpu')
model.config._attn_implementation = 'sdpa'
model.eval()
output = model(input_ids=input_ids)
output.logits

tensor([[[ -6.8392,   8.6455,  -7.1738,  ...,  -1.4409,  -1.0301,  -1.7648],
         [ -9.0968,   4.3056,  -9.1296,  ...,  -1.4099,  -1.9358,  -2.9704],
         [ -8.7057,   6.1272,  -9.4507,  ...,  -1.4659,  -2.0700,  -4.0830],
         ...,
         [-10.8051,   7.4513, -11.5286,  ...,  -6.0487,  -3.2882,  -3.9321],
         [ -9.4184,   6.6918, -10.2852,  ...,  -5.2178,  -2.8604,  -4.1874],
         [-10.2871,   8.5075, -11.0755,  ...,  -5.4560,  -2.9234,  -5.3958]]],
       grad_fn=<UnsafeViewBackward0>)

In [58]:
tokenizer = transformers.AutoTokenizer.from_pretrained("Fazzioni/tokenizer-fineweb.50M-32k")

In [79]:
text= ' O brasil é um pais '
input_ids = tokenizer(text, return_tensors='pt').input_ids
input_ids

tensor([[   1,  355, 1276,  298,  299, 2907,  179]])

In [81]:
output = model.generate(input_ids, max_new_tokens=200, do_sample=True, top_p=0.9, temperature=0.8, top_k=65)

In [82]:
tokenizer.decode(output[0], skip_special_tokens=True)

' O brasil é um pais de enorme desigualdade social, a desigualdade social esta presente em todas as esferas da sociedade, isso e claro, se reflete no sistema de educacao. Mas uma educacao de qualidade e a base para uma sociedade justa. A educacao de qualidade e fundamental para a sociedade brasileira, pois a educacao de qualidade nao garante a todos o acesso a educacao, ela garante o acesso as universidades e cursos profissionalizantes, cursos que o governo e as empresas privadas oferecem, e o acesso a um ensino de qualidade. A educacao de qualidade nao pode ser a unica solucao para a crise, pois ela tem que ser parte do processo de desenvolvimento, por isso e preciso investir no sistema de educacao, e necessario investir em recursos humanos, materiais e tecnologicos para melhorar o ensino. A educacao de qualidade e necessaria para todos os brasileiros, nao adianta somente a escola de qualidade,'

In [84]:
# number os parameters
sum(p.numel() for p in model.parameters())

345653120